In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from keras.layers import Dense, Conv2D, LeakyReLU, Flatten, Dropout,GlobalAveragePooling2D, Input
from keras.models import Model, Sequential
from keras.applications import EfficientNetV2L
from keras.applications.xception import Xception
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental import preprocessing

In [12]:
ds_train, ds_test = tfds.load('stanford_dogs', split=['train', "test"], as_supervised=True, shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Shuffling /Users/chinedu/tensorflow_datasets/stanford_dogs/0.2.0.incompletePPLSHJ/stanford_dogs-train.tfrecord…

Generating test examples...:   0%|          | 0/8580 [00:00<?, ? examples/s]

Shuffling /Users/chinedu/tensorflow_datasets/stanford_dogs/0.2.0.incompletePPLSHJ/stanford_dogs-test.tfrecord*…

Dataset stanford_dogs downloaded and prepared to /Users/chinedu/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.
Metal device set to: Apple M2


2022-10-07 21:06:04.510803: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-07 21:06:04.512185: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
def normalize_img(image, label):
  image = tf.image.resize(image, size=(256, 256))
  return image, label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [14]:
data_augmentation = Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [25]:
base_model = EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(Input(shape=(256,256,3)))
# model.add(data_augmentation)
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(120,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 8, 8, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 120)               153720    
                                                                 
Total params: 117,900,568
Trainable params: 153,720
Non-trainable params: 117,746,848
_________________________________________________________________


In [26]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [27]:
checkpoint = ModelCheckpoint(
    './base.model',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    save_freq="epoch"
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True,
    mode='auto'
)
reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)
callbacks = [checkpoint, earlystop]

In [ ]:
model.fit(ds_train, epochs=200,
          validation_data=ds_test,
          callbacks=callbacks)

Epoch 1/200


2022-10-07 21:19:59.126469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - ETA: 0s - loss: 2.9204 - accuracy: 0.6568

2022-10-07 21:27:20.333824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 0.97962, saving model to ./base.model


...

INFO:tensorflow:Assets written to: ./base.model/assets


94/94 [==============================] - 1021s 11s/step - loss: 2.9204 - accuracy: 0.6568 - val_loss: 0.9796 - val_accuracy: 0.9273
Epoch 2/200
94/94 [==============================] - ETA: 0s - loss: 1.1144 - accuracy: 0.8260
Epoch 2: val_loss improved from 0.97962 to 0.40019, saving model to ./base.model


INFO:tensorflow:Assets written to: ./base.model/assets


INFO:tensorflow:Assets written to: ./base.model/assets


94/94 [==============================] - 1067s 11s/step - loss: 1.1144 - accuracy: 0.8260 - val_loss: 0.4002 - val_accuracy: 0.9303
Epoch 3/200
94/94 [==============================] - ETA: 0s - loss: 0.8573 - accuracy: 0.8373